In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
# %conda install pyodbc -y
# %conda install scikit-learn
# %conda install seaborn

In [5]:
get_current_time_ist()

'20241124021837PM'

In [6]:
show_dateinfo()

+--------------------+------------+
|       index        |   Value    |
+--------------------+------------+
|    date current    | 2024-11-24 |
|   date lag (-1)    | 2024-10-31 |
| month name current |  November  |
|   month name lag   |  October   |
|    year current    |    2024    |
|      year lag      |    2024    |
|   mmyyyy_current   |   112024   |
|   yyyymm_current   |   202411   |
|     mmyyyy_lag     |   102024   |
|     yyyymm_lag     |   202410   |
+--------------------+------------+


In [7]:
df = pd.read_csv('sample_datasets/BankChurners.csv')

In [8]:
df.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'],
      dtype='object')

## correlation droppings

In [9]:
numerical_df = df.select_dtypes(include='number')

In [10]:
corr_df = numerical_df.corr().abs()

In [11]:
from utilities import iteratively_remove_correlated_features,analyse_feature,woe_iv,iteratively_remove_correlated_features_with_target

In [12]:
drop_features,log_df_target = iteratively_remove_correlated_features_with_target(corr_df,'Dependent_count')

+-------+---------------------+-----------------------+-----------------+-----------------+-------------+-----------------------+
| index |       Feature1      |        Feature2       | Feature1_Target | Feature2_Target | Correlation |        Dropping       |
+-------+---------------------+-----------------------+-----------------+-----------------+-------------+-----------------------+
|   0   |     Customer_Age    |     Months_on_book    |      0.076      |      0.057      |    0.965    |     Months_on_book    |
|   1   |     Credit_Limit    |    Avg_Open_To_Buy    |      -0.09      |      -0.093     |    0.999    |      Credit_Limit     |
|   2   |   Total_Trans_Amt   |     Total_Trans_Ct    |      -0.197     |      -0.149     |    0.956    |     Total_Trans_Ct    |
|   3   |     Credit_Limit    | Avg_Utilization_Ratio |      -0.09      |      -0.183     |    0.628    |      Credit_Limit     |
|   4   | Total_Revolving_Bal | Avg_Utilization_Ratio |      -0.202     |      -0.183     

In [13]:
all_features_dropped, log_df = iteratively_remove_correlated_features(corr_df)

+-------+---------------------+-----------------------+-------------------+-------------------+-------------+-----------------------+
| index |       Feature1      |        Feature2       | Feature1_Variance | Feature2_Variance | Correlation |        Dropping       |
+-------+---------------------+-----------------------+-------------------+-------------------+-------------+-----------------------+
|   0   |     Customer_Age    |     Months_on_book    |        0.1        |       0.101       |    0.965    |      Customer_Age     |
|   1   |     Credit_Limit    |    Avg_Open_To_Buy    |       0.126       |       0.129       |    0.999    |      Credit_Limit     |
|   2   |   Total_Trans_Amt   |     Total_Trans_Ct    |       0.093       |       0.095       |    0.956    |    Total_Trans_Amt    |
|   3   |     Credit_Limit    | Avg_Utilization_Ratio |       0.126       |       0.099       |    0.628    | Avg_Utilization_Ratio |
|   4   | Total_Revolving_Bal | Avg_Utilization_Ratio |       

In [14]:
all_features_dropped

['Avg_Utilization_Ratio',
 'Customer_Age',
 'Total_Trans_Amt',
 'Credit_Limit',
 'Total_Revolving_Bal']

In [15]:
log_df.columns

Index(['Feature1', 'Feature2', 'Feature1_Variance', 'Feature2_Variance',
       'Correlation', 'Dropping', 'Iteration_Step',
       'Feature1_Correlated_With_Count', 'Feature2_Correlated_With_Count',
       'Feature1_Correlated_With_List', 'Feature2_Correlated_With_List'],
      dtype='object')

## feature analysis

In [16]:
analyse_feature(df,'Attrition_Flag','Gender',relation='cat2cat',bins='False')

feature analysis output is a dictionary with keys counts,total_ratio,bin_ratio
Counts
+-------------------+------+------+-------+
|   Attrition_Flag  |  F   |  M   | total |
+-------------------+------+------+-------+
| Attrited Customer | 930  | 697  |  1627 |
| Existing Customer | 4428 | 4072 |  8500 |
|       total       | 5358 | 4769 | 10127 |
+-------------------+------+------+-------+
Total Ratio
+-------------------+--------+--------+---------+
|   Attrition_Flag  |   F    |   M    |  total  |
+-------------------+--------+--------+---------+
| Attrited Customer | 9.18%  | 6.88%  |  16.07% |
| Existing Customer | 43.72% | 40.21% |  83.93% |
|       total       | 52.91% | 47.09% | 100.00% |
+-------------------+--------+--------+---------+
Binwise Ratio
+-------------------+--------+--------+---------+
|   Attrition_Flag  |   F    |   M    |  total  |
+-------------------+--------+--------+---------+
| Attrited Customer | 57.16% | 42.84% | 100.00% |
| Existing Customer | 52.09% |

In [18]:
analyse_feature(df,'Attrition_Flag','Avg_Utilization_Ratio',relation='cat2con',bins='False')

+-------------------+--------+-------+-------+-----+-------+-------+-------+-------+--------+
|   Attrition_Flag  | count  |  mean |  std  | min |  25%  |  50%  |  75%  |  max  | ratio  |
+-------------------+--------+-------+-------+-----+-------+-------+-------+-------+--------+
| Attrited Customer | 1627.0 | 0.162 | 0.264 | 0.0 |  0.0  |  0.0  | 0.231 | 0.999 | 16.07% |
| Existing Customer | 8500.0 | 0.296 | 0.273 | 0.0 | 0.055 | 0.211 | 0.529 | 0.994 | 83.93% |
+-------------------+--------+-------+-------+-----+-------+-------+-------+-------+--------+


In [19]:
analyse_feature(df,'Customer_Age','Avg_Utilization_Ratio',relation='con2con',bins='False')

+-------------------+--------+-------+-------+-----+-------+-------+-------+-------+--------+
| Customer_Age_bins | count  |  mean |  std  | min |  25%  |  50%  |  75%  |  max  | ratio  |
+-------------------+--------+-------+-------+-----+-------+-------+-------+-------+--------+
|   (25.999, 36.0]  | 1140.0 | 0.305 | 0.281 | 0.0 | 0.036 | 0.242 | 0.542 |  0.99 | 11.26% |
|    (36.0, 39.0]   | 896.0  | 0.281 | 0.277 | 0.0 | 0.026 | 0.185 | 0.508 | 0.999 | 8.85%  |
|    (39.0, 42.0]   | 1166.0 | 0.264 | 0.273 | 0.0 |  0.0  | 0.159 | 0.481 | 0.972 | 11.51% |
|    (42.0, 44.0]   | 973.0  | 0.244 | 0.266 | 0.0 |  0.0  | 0.141 | 0.424 | 0.983 | 9.61%  |
|    (44.0, 46.0]   | 976.0  | 0.264 | 0.274 | 0.0 | 0.021 | 0.144 | 0.493 | 0.985 | 9.64%  |
|    (46.0, 48.0]   | 951.0  |  0.26 | 0.272 | 0.0 |  0.0  | 0.155 | 0.465 | 0.978 | 9.39%  |
|    (48.0, 51.0]   | 1345.0 | 0.252 | 0.267 | 0.0 |  0.0  |  0.14 | 0.463 | 0.984 | 13.28% |
|    (51.0, 53.0]   | 763.0  | 0.293 | 0.281 | 0.0 | 0.035 |

In [20]:
analyse_feature(df,'Customer_Age','Attrition_Flag',relation='con2cat',bins='False')

feature analysis output is a dictionary with keys counts,total_ratio,bin_ratio
Counts
+-------------------+-------------------+-------------------+-------+
| Customer_Age_bins | Existing Customer | Attrited Customer | total |
+-------------------+-------------------+-------------------+-------+
|   (25.999, 36.0]  |        994        |        146        |  1140 |
|    (36.0, 39.0]   |        764        |        132        |  896  |
|    (39.0, 42.0]   |        964        |        202        |  1166 |
|    (42.0, 44.0]   |        804        |        169        |  973  |
|    (44.0, 46.0]   |        815        |        161        |  976  |
|    (46.0, 48.0]   |        790        |        161        |  951  |
|    (48.0, 51.0]   |        1137       |        208        |  1345 |
|    (51.0, 53.0]   |        646        |        117        |  763  |
|    (53.0, 57.0]   |        875        |        196        |  1071 |
|    (57.0, 73.0]   |        711        |        135        |  846  |
|   

## woe

In [23]:
woe_iv_df = woe_iv(df,'Customer_Age','Attrition_Flag','Attrited Customer',bins='False',continuous=True)

+-------------------+-----------------------+-------------------------+---------+------------+------------+--------+-------+
| Customer_Age_bins | Attrited Customer_sum | Attrited Customer_count | %events | #nonevents | %nonevents |  woe   |   IV  |
+-------------------+-----------------------+-------------------------+---------+------------+------------+--------+-------+
|   (25.999, 36.0]  |          146          |           1140          |   0.09  |    994     |   0.117    | -0.262 | 0.007 |
|    (36.0, 39.0]   |          132          |           896           |  0.081  |    764     |    0.09    | -0.105 | 0.001 |
|    (39.0, 42.0]   |          202          |           1166          |  0.124  |    964     |   0.113    | 0.093  | 0.001 |
|    (42.0, 44.0]   |          169          |           973           |  0.104  |    804     |   0.095    | 0.091  | 0.001 |
|    (44.0, 46.0]   |          161          |           976           |  0.099  |    815     |   0.096    | 0.031  |  0.0  |


In [24]:
woe_iv_df2 = woe_iv(df,'Gender','Attrition_Flag','Attrited Customer',bins='False',continuous=False)

+--------+-----------------------+-------------------------+---------+------------+------------+--------+-------+
| Gender | Attrited Customer_sum | Attrited Customer_count | %events | #nonevents | %nonevents |  woe   |   IV  |
+--------+-----------------------+-------------------------+---------+------------+------------+--------+-------+
|   F    |          930          |           5358          |  0.572  |    4428    |   0.521    | 0.093  | 0.005 |
|   M    |          697          |           4769          |  0.428  |    4072    |   0.479    | -0.113 | 0.006 |
+--------+-----------------------+-------------------------+---------+------------+------------+--------+-------+


## chi-square selection

In [28]:
from utilities import chi_selection,entropy

In [26]:
table(chi_selection(df,'Avg_Utilization_Ratio'))

Error processing feature Unnamed: 20: index 0 is out of bounds for axis 0 with size 0
Error processing feature Unnamed: 22: index 0 is out of bounds for axis 0 with size 0
+------------------------------+------------------------+--------------------+
|           feature            |        p_value         |  chi_square_stats  |
+------------------------------+------------------------+--------------------+
|  Months_Inactive_12_mon_bin  | 0.0009122423017769385  | 3131.7880543128026 |
|  Contacts_Count_12_mon_bin   |   0.9202920802972726   | 3729.162869890826  |
|     Dependent_count_bin      |   0.5587963729847766   | 3838.3660987260946 |
| Total_Relationship_Count_bin |   0.5040137398566246   | 3850.4504211954163 |
|      Months_on_book_bin      |   0.3112731284295795   | 6797.647239342002  |
|       Customer_Age_bin       |   0.8399867478542998   | 8536.075795687198  |
|   Total_Amt_Chng_Q4_Q1_bin   |   0.4384178777242985   | 8686.753122264443  |
|   Total_Ct_Chng_Q4_Q1_bin    |   0.3

## Entropy

In [29]:
entropy('Customer_Age')

0.298746875060096

In [30]:
entropy('Total_Amt_Chng_Q4_Q1')

0.21609640474436778